In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
print("TensorFlow version:", tf.__version__)
# Load the TensorBoard notebook extension

import datetime
import shutil

TensorFlow version: 2.11.0


In [15]:

from tensorflow.keras import Model
from pathlib import Path
import pandas as pd

In [16]:
BATCH_SIZE = 64
BUFFER_SIZE = 128
LEARNING_RATE = 0.05
EPOCHS = 50

In [17]:
input_dir = Path('./data/prepared')
logs_path = Path('./data/logs')
if logs_path.exists():
  shutil.rmtree(logs_path)
logs_path.mkdir(parents=True)

X_train_name = input_dir / 'X_train.csv'
y_train_name = input_dir / 'y_train.csv'
X_test_name = input_dir / 'X_test.csv'
y_test_name = input_dir / 'y_test.csv'

X_train = pd.read_csv(X_train_name)
y_train = pd.read_csv(y_train_name)
X_test = pd.read_csv(X_test_name)
y_test = pd.read_csv(y_test_name)

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [18]:
X_train.shape

(1961, 25)

In [19]:
class SomeModel(Model):
  def __init__(self, neurons_cnt=128):
    super(SomeModel, self).__init__()

    self.d_in = Dense(25, activation='relu')
    self.d1 = Dense(neurons_cnt, activation='relu')
    self.d2 = Dense(neurons_cnt, activation='relu')
    self.d_out = Dense(1)

  def call(self, x):
    x = self.d_in(x)
    x = self.d1(x)
    return self.d_out(x)

In [20]:
# Create an instance of the model
model = SomeModel()

In [21]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE)

In [22]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanAbsoluteError(name='train_mae')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanAbsoluteError(name='test_mae')

In [23]:
@tf.function
def train_step(input_vector, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(input_vector, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [24]:
@tf.function
def test_step(input_vector, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(input_vector, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [25]:
from tensorflow import keras

In [26]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = logs_path / 'gradient_tape' / current_time / 'train'
train_log_dir.mkdir(exist_ok=True, parents=True)
test_log_dir = logs_path / 'gradient_tape' / current_time / 'test'
test_log_dir.mkdir(exist_ok=True, parents=True)
train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
test_summary_writer = tf.summary.create_file_writer(str(test_log_dir))

logdir=logs_path / "fit" / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir.mkdir(exist_ok=True, parents=True)
fit_summary_writer = tf.summary.create_file_writer(str(logdir))

tf.summary.trace_on(graph=True, profiler=True)
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  for (x_train, y_train) in train_ds:

    with fit_summary_writer.as_default():
      train_step(x_train, y_train)


  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_ds:
    test_step(x_test, y_test)

  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('mae', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test MAE: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result(),
                         test_loss.result(),
                         test_accuracy.result()))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

with fit_summary_writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir
  )

2022-12-19 16:01:56.401459: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2022-12-19 16:01:56.401467: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.


Epoch 1, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 2, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 3, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 4, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 5, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 6, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 7, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 8, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 9, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 10, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 11, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 12, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 13, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 14, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 15, Loss: nan, Accuracy: nan, Test Loss: nan, Test MAE: nan
Epoch 16, Loss: nan

2022-12-19 16:01:57.900876: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-12-19 16:01:58.424918: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
